In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

c:\Users\Admin\OneDrive\Python


In [2]:
import numpy as np
import pandas as pd
import scipy
import toolkit as ftk
import statsmodels.api as sm
from sklearn import linear_model

unit_price = pd.Series([100,100.3,102.91,104.04,103.1,104.55,107.06,108.66,115.83,114.21,118.67,118.07,127.64,132.74,127.83,120.03,121.71,115.75,113.32,120.34,127.32,119.18,121.2,120.72,120.48,117.95,119.24,124.85,127.84,132.06,131.14,137.3,138.13,139.51,139.23,143.96,145.4],
    index=pd.date_range('1999-12', periods=37, freq='M'))
benchmark_price = pd.Series([100,100.2,102.71,104.55,103.4,104.85,107.26,108.76,115.83,114.1,118.89,118.53,128.37,133.38,128.31,120.35,122.16,116.29,113.97,120.81,127.57,119.03,121.29,120.92,120.8,117.66,118.48,123.58,127.16,132,131.73,138.45,140.39,142.21,142.64,147.49,150.59],
    index=pd.date_range('1999-12', periods=37, freq='M'))
risk_free_return = pd.Series([0.001,0.001,0.002,0.002,0.002,0.002,0.002,0.003,0.003,0.004,0.004,0.003,0.003,0.003,0.003,0.004,0.002,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.002,0.002,0.002,0.002,0.002,0.002,0.002],
    index=pd.period_range('2000-01', periods=36, freq='M'))

aa = ftk.price_to_return(unit_price)
bb = ftk.price_to_return(benchmark_price)

prices = pd.concat([unit_price, benchmark_price], axis=1)
benchmarks = prices = pd.concat([benchmark_price, unit_price], axis=1)




0.007946014194400774